# Gathers phenotypic information for basic analysis

In [1]:
import pandas as pd
import numpy as np
from bids import BIDSLayout, BIDSLayoutIndexer
bids_path = "/home/lukeh/hpcworking/lukeH/projects/QNC_outcomes/data/bids/"

# get subjects from bids 
indexer = BIDSLayoutIndexer(validate=False, index_metadata=False)
layout = BIDSLayout(bids_path, indexer=indexer)
subject_ids = layout.get_subjects()

# read in clinical data
file = '../../data/11092024 Anonymised QNC Clinical Data.xlsx'
df = pd.read_excel(file, 'Coordinates')

# reorganise coordinates
df_list = []
for subj in subject_ids:
    subj_data = df[df.Participant_Number.str.contains(subj)]
    bad_subj = 1

    if len(subj_data) == 1:
        x = subj_data.iloc[0,1]
        y = subj_data.iloc[0,2]
        z = subj_data.iloc[0,3]
        bad_subj = 0

    elif len(subj_data) == 2:
        subj_data = df[df.Participant_Number.str.contains(subj+"M")]
        if len(subj_data) == 1:
            x = subj_data.iloc[0,1]
            y = subj_data.iloc[0,2]
            z = subj_data.iloc[0,3]
            bad_subj = 0

    if bad_subj == 1:
        print(subj)
        print(subj_data)
        x = 0
        y = 0
        z = 0

    row = pd.DataFrame()
    row["participant_id"] = ["sub-"+str(subj)]
    row["x"] = x
    row["y"] = y
    row["z"] = z
    df_list.append(row)
out_df = pd.concat(df_list)

# Add MADRS score
df = pd.read_excel(file, 'MADRS')
out_df["change_in_MADRS"] = np.nan
for subj in subject_ids:
    subj_data = df[df.participant_id.str.contains(subj)]
    if len(subj_data) == 2:
        score = subj_data.loc[subj_data.session == "post", "total"].values - subj_data.loc[subj_data.session == "pre", "total"].values
    else:
        print(subj)
        score = "n/a"
    out_df.loc[out_df.participant_id == "sub-"+subj, "change_in_MADRS"] = score

# add tier
df = pd.read_excel(file, 'participant_information')
out_df["tier"] = np.nan
for subj in subject_ids:
    subj_data = df[df.participant_id.str.contains(subj)]
    subj_data = subj_data.research_tier.to_list()
    
    if len(subj_data) == 2:
        subj_data = df[df.participant_id.str.contains(subj+"M")]
        subj_data = subj_data.research_tier.to_list()

    if len(subj_data) == 1:
        if subj_data[0] == "RCT acceptable":
            tier = 1
            
        elif subj_data[0] == "naturalistic":
            tier = 2

        elif subj_data[0] == "bipolar or neurological disorder":
            tier = 3

        else:
            tier = "n/a"
            print(subj)
    else:
        tier = "n/a"
        print(subj)
        print(subj_data)
        break
    out_df.loc[out_df.participant_id == "sub-"+subj, "tier"] = tier

159
159


/tmp/ipykernel_2100791/891824622.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'n/a' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  out_df.loc[out_df.participant_id == "sub-"+subj, "change_in_MADRS"] = score
/tmp/ipykernel_2100791/891824622.py:91: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'n/a' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  out_df.loc[out_df.participant_id == "sub-"+subj, "tier"] = tier


In [2]:
out_df.to_csv(bids_path+'phenotype/coordinates.tsv', index=False, sep="\t")
out_df.head(100)

,participant_id,x,y,z,change_in_MADRS,tier
0,sub-092,-36.0,40.0,38.0,-25.0,2.0
0,sub-093,-42.0,42.0,28.0,-29.0,2.0
0,sub-094,-44.0,42.0,28.0,-17.0,1.0
0,sub-097,-30.0,42.0,40.0,-5.0,2.0
0,sub-098,-36.0,30.0,36.0,-32.0,2.0
0,sub-099,-34.0,42.0,28.0,-3.0,2.0
0,sub-103,-38.0,34.0,36.0,-12.0,1.0
0,sub-105,-40.0,32.0,40.0,-26.0,1.0
0,sub-106,-34.0,38.0,32.0,-29.0,1.0
0,sub-108,-38.0,0.0,56.0,-23.0,2.0


# Connie updated coordinates

In [5]:
df = pd.read_excel("../../data/modified_13112024_QNC_coords.xlsx")
df.head()

,Participant_Number,Predicted (MNI)\n x,Predicted (MNI)\ny,Predicted (MNI)\nz,Predicted (Native)\n x,Predicted (Native)\ny,Predicted (Native)\n z,Treatment (MNI)\n x,Treatment (MNI)\ny,Treatment (MNI)\nz,...,Simon_Treatment (MNI)\ny,Simon_Treatment (MNI)\nz,Transformed_Nlin6Asym_Treatment_MNI_X,Transformed_Nlin6Asym_Treatment_MNI_Y,Transformed_Nlin6Asym_Treatment_MNI_Z,Treatment (Native) \nx,Treatment (Native) \ny,Treatment (Native) \nz,Connie_conversion_notes,Notes_Treatment Coordinates
0,049,-48,30,30,NaN,NaN,NaN,-48.0,30.0,30.0,...,30.0,30.0,-48.3581,37.4320,21.55560,NaN,NaN,NaN,NaN,NaN
1,051,-48,34,30,NaN,NaN,NaN,-45.0,31.0,27.0,...,31.0,27.0,-48.5007,36.8633,25.30820,NaN,NaN,NaN,NaN,Amended before treatment from cluster centre t...
2,053,-44,44,14,NaN,NaN,NaN,-44.0,44.0,14.0,...,44.0,14.0,-48.0532,46.8030,3.60871,NaN,NaN,NaN,NaN,NaN
3,055,-28,48,34,NaN,NaN,NaN,-28.0,48.0,34.0,...,48.0,34.0,-28.9372,59.6855,15.88680,NaN,NaN,NaN,NaN,NaN
4,058,-38,30,18,NaN,NaN,NaN,-38.0,33.0,39.0,...,33.0,39.0,-40.2376,43.9234,34.33490,NaN,NaN,NaN,NaN,Amended before treatment from cluster centre t...


In [7]:
out_df["Transformed_Nlin6Asym_Treatment_MNI_X"] = np.nan

for subj in subject_ids:
    subj_data = df[df.Participant_Number.str.contains(subj)]
    bad_subj = 1

    if len(subj_data) == 1:
        x = subj_data["Transformed_Nlin6Asym_Treatment_MNI_X"].values[0]
        y = subj_data["Transformed_Nlin6Asym_Treatment_MNI_Y"].values[0]
        z = subj_data["Transformed_Nlin6Asym_Treatment_MNI_Z"].values[0]
        bad_subj = 0

    elif len(subj_data) == 2:
        subj_data = df[df.Participant_Number.str.contains(subj+"M")]
        if len(subj_data) == 1:
            x = subj_data["Transformed_Nlin6Asym_Treatment_MNI_X"].values[0]
            y = subj_data["Transformed_Nlin6Asym_Treatment_MNI_Y"].values[0]
            z = subj_data["Transformed_Nlin6Asym_Treatment_MNI_Z"].values[0]
            bad_subj = 0

    if bad_subj == 1:
        print(subj)
        #print(subj_data)
        x = 0
        y = 0
        z = 0

    # add to df
    out_df.loc[out_df.participant_id == "sub-"+str(subj), "Transformed_Nlin6Asym_Treatment_MNI_X"] = x
    out_df.loc[out_df.participant_id == "sub-"+str(subj), "Transformed_Nlin6Asym_Treatment_MNI_Y"] = y
    out_df.loc[out_df.participant_id == "sub-"+str(subj), "Transformed_Nlin6Asym_Treatment_MNI_Z"] = z


159


In [8]:
out_df.to_csv(bids_path+'phenotype/coordinates.tsv', index=False, sep="\t")
out_df.head(100)

,participant_id,x,y,z,change_in_MADRS,tier,Transformed_Nlin6Asym_Treatment_MNI_X,Transformed_Nlin6Asym_Treatment_MNI_Y,Transformed_Nlin6Asym_Treatment_MNI_Z
0,sub-092,-36.0,40.0,38.0,-25.0,2.0,-39.7742,47.97380,31.68350
0,sub-093,-42.0,42.0,28.0,-29.0,2.0,-47.1543,43.11230,28.53000
0,sub-094,-44.0,42.0,28.0,-17.0,1.0,-50.8154,43.27570,11.62400
0,sub-097,-30.0,42.0,40.0,-5.0,2.0,-30.6853,47.70400,31.00060
0,sub-098,-36.0,30.0,36.0,-32.0,2.0,-44.6883,40.28540,20.17470
0,sub-099,-34.0,42.0,28.0,-3.0,2.0,-39.3510,50.18480,22.25630
0,sub-103,-38.0,34.0,36.0,-12.0,1.0,-43.2435,37.17620,33.29140
0,sub-105,-40.0,32.0,40.0,-26.0,1.0,-41.4525,34.99170,42.19430
0,sub-106,-34.0,38.0,32.0,-29.0,1.0,-34.1798,44.90390,25.63820
0,sub-108,-38.0,0.0,56.0,-23.0,2.0,-37.4534,55.18800,17.20310


# Check coords in relation to useable brain data

In [9]:
from nilearn.maskers import NiftiSpheresMasker
from nilearn import datasets, plotting
from nilearn.masking import _unmask_3d
from nilearn.maskers import nifti_spheres_masker
import nibabel as nib
from nibabel import Nifti1Image
from tqdm import tqdm


radius_5_total = 81
radius_10_total = 515

# add inidividualised seed
coord_df = out_df.copy()
results = []
for subj in tqdm(coord_df.participant_id.unique()):
    for radius in [5, 10]:
        for ses in ["pre", "post"]:

            sid = subj.split("-")[-1]

            row = coord_df[coord_df.participant_id == "sub-"+sid]

            # Connie coordinate
            target = []
            target.append(np.array((row.Transformed_Nlin6Asym_Treatment_MNI_X.values[0],
                                    row.Transformed_Nlin6Asym_Treatment_MNI_Y.values[0],
                                    row.Transformed_Nlin6Asym_Treatment_MNI_Z.values[0])))


            # get mask img
            input_path = f"/home/lukeh/hpcworking/lukeH/projects/QNC_outcomes/data/derivatives/fmriprep_24.0.1/sub-{sid}/ses-{ses}/func/"
            input_img = f"{input_path}sub-{sid}_ses-{ses}_task-rest_space-MNI152NLin6Asym_res-02_desc-brain_mask.nii.gz"

            # let's assume we are in MNI space
            #brain_mask = datasets.load_mni152_brain_mask()
            brain_mask = nib.load(input_img)
            _, A = nifti_spheres_masker._apply_mask_and_get_affinity(
                seeds=target,
                niimg=None,
                radius=radius,
                allow_overlap=False, 
                mask_img=brain_mask)

            sphere_mask = _unmask_3d(
                X=A.toarray().flatten(), 
                mask=brain_mask.get_fdata().astype(bool))

            sphere_mask = Nifti1Image(sphere_mask, brain_mask.affine)

            sum_vals = np.sum(sphere_mask.get_fdata())

            if radius == 5:
                sum_vals = (sum_vals / radius_5_total) * 100
            elif radius == 10:
                sum_vals = (sum_vals / radius_10_total) * 100

            _df = pd.DataFrame()
            _df['participant_id'] = [subj]
            _df["radius"] = radius
            _df["session"] = ses
            _df["perc. voxels in brain mask"] = sum_vals
            results.append(_df)
results = pd.concat(results)
results.head()

100%|██████████| 29/29 [02:46<00:00,  5.75s/it]


,participant_id,radius,session,perc. voxels in brain mask
0,sub-092,5,pre,24.691358
0,sub-092,5,post,39.506173
0,sub-092,10,pre,38.252427
0,sub-092,10,post,42.330097
0,sub-093,5,pre,2.469136


In [12]:
results.head()

AttributeError: 'list' object has no attribute 'head'

In [10]:
from nilearn.maskers import NiftiSpheresMasker
from nilearn import datasets, plotting
from nilearn.masking import _unmask_3d
from nilearn.maskers import nifti_spheres_masker
import nibabel as nib
from nibabel import Nifti1Image
from tqdm import tqdm


radius = 10
ses = "pre"
perc_cutoff = 90
# add inidividualised seed
coord_df = out_df.copy()
results = []
for subj in tqdm(coord_df.participant_id.unique()):

    sid = subj.split("-")[-1]

    row = coord_df[coord_df.participant_id == "sub-"+sid]

    # Connie coordinate
    target = []
    target.append(np.array((row.Transformed_Nlin6Asym_Treatment_MNI_X.values[0],
                            row.Transformed_Nlin6Asym_Treatment_MNI_Y.values[0],
                            row.Transformed_Nlin6Asym_Treatment_MNI_Z.values[0])))
    print(target)

    # get mask img
    input_path = f"/home/lukeh/hpcworking/lukeH/projects/QNC_outcomes/data/derivatives/fmriprep_24.0.1/sub-{sid}/ses-{ses}/func/"
    input_img = f"{input_path}sub-{sid}_ses-{ses}_task-rest_space-MNI152NLin6Asym_res-02_desc-brain_mask.nii.gz"

    # let's assume we are in MNI space
    #brain_mask = datasets.load_mni152_brain_mask()
    brain_mask = nib.load(input_img)
    _, A = nifti_spheres_masker._apply_mask_and_get_affinity(
        seeds=target,
        niimg=None,
        radius=radius,
        allow_overlap=False, 
        mask_img=brain_mask)

    sphere_mask = _unmask_3d(
        X=A.toarray().flatten(), 
        mask=brain_mask.get_fdata().astype(bool))

    sphere_mask = Nifti1Image(sphere_mask, brain_mask.affine)

    sum_vals = np.sum(sphere_mask.get_fdata())

    if radius == 5:
        sum_vals = (sum_vals / radius_5_total) * 100
    elif radius == 10:
        sum_vals = (sum_vals / radius_10_total) * 100
    print(sum_vals)

    while sum_vals < perc_cutoff:
        target[0][0] = target[0][0] + 1
        target[0][1] = target[0][1] - 1
        target[0][2] = target[0][2] - 1
        print(target)


        _, A = nifti_spheres_masker._apply_mask_and_get_affinity(
            seeds=target,
            niimg=None,
            radius=radius,
            allow_overlap=False, 
            mask_img=brain_mask)

        sphere_mask = _unmask_3d(
            X=A.toarray().flatten(), 
            mask=brain_mask.get_fdata().astype(bool))

        sphere_mask = Nifti1Image(sphere_mask, brain_mask.affine)

        sum_vals = np.sum(sphere_mask.get_fdata())

        if radius == 5:
            sum_vals = (sum_vals / radius_5_total) * 100
        elif radius == 10:
            sum_vals = (sum_vals / radius_10_total) * 100
        print(sum_vals)
    out_df.loc[(out_df.participant_id==subj), "Transformed_Nlin6Asym_Treatment_MNI_X_adj10"] = target[0][0]
    out_df.loc[(out_df.participant_id==subj), "Transformed_Nlin6Asym_Treatment_MNI_Y_adj10"] = target[0][1]
    out_df.loc[(out_df.participant_id==subj), "Transformed_Nlin6Asym_Treatment_MNI_Z_adj10"] = target[0][2]

  0%|          | 0/29 [00:00<?, ?it/s]

[array([-39.7742,  47.9738,  31.6835])]
38.25242718446602
[array([-38.7742,  46.9738,  30.6835])]
51.84466019417475
[array([-37.7742,  45.9738,  29.6835])]
63.689320388349515
[array([-36.7742,  44.9738,  28.6835])]
76.50485436893204
[array([-35.7742,  43.9738,  27.6835])]
86.01941747572816
[array([-34.7742,  42.9738,  26.6835])]


  3%|▎         | 1/29 [00:08<03:52,  8.30s/it]

96.11650485436894
[array([-47.1543,  43.1123,  28.53  ])]
26.990291262135923
[array([-46.1543,  42.1123,  27.53  ])]
37.86407766990291
[array([-45.1543,  41.1123,  26.53  ])]
51.45631067961165
[array([-44.1543,  40.1123,  25.53  ])]
62.912621359223294
[array([-43.1543,  39.1123,  24.53  ])]
75.92233009708738
[array([-42.1543,  38.1123,  23.53  ])]
85.43689320388349
[array([-41.1543,  37.1123,  22.53  ])]


  7%|▋         | 2/29 [00:18<04:06,  9.13s/it]

...

96.11650485436894
[array([-41.4411,  43.5734,  24.3964])]
87.37864077669903
[array([-40.4411,  42.5734,  23.3964])]


 97%|█████████▋| 28/29 [02:17<00:02,  2.62s/it]

96.11650485436894
[array([0., 0., 0.])]


100%|██████████| 29/29 [02:19<00:00,  4.80s/it]

100.0


In [11]:
# from nilearn.maskers import NiftiSpheresMasker
# from nilearn import datasets, plotting
# from nilearn.masking import _unmask_3d
# from nilearn.maskers import nifti_spheres_masker
# import nibabel as nib
# from nibabel import Nifti1Image
# from tqdm import tqdm


# radius = 5
# ses = "pre"

# # add inidividualised seed
# coord_df = out_df.copy()
# results = []
# for subj in tqdm(coord_df.participant_id.unique()):

#     sid = subj.split("-")[-1]

#     row = coord_df[coord_df.participant_id == "sub-"+sid]

#     # Connie coordinate
#     target = []
#     target.append(np.array((row.Transformed_Nlin6Asym_Treatment_MNI_X.values[0],
#                             row.Transformed_Nlin6Asym_Treatment_MNI_Y.values[0],
#                             row.Transformed_Nlin6Asym_Treatment_MNI_Z.values[0])))
#     print(target)

#     # get mask img
#     input_path = f"/home/lukeh/hpcworking/lukeH/projects/QNC_outcomes/data/derivatives/fmriprep_24.0.1/sub-{sid}/ses-{ses}/func/"
#     input_img = f"{input_path}sub-{sid}_ses-{ses}_task-rest_space-MNI152NLin6Asym_res-02_desc-brain_mask.nii.gz"

#     # let's assume we are in MNI space
#     #brain_mask = datasets.load_mni152_brain_mask()
#     brain_mask = nib.load(input_img)
#     _, A = nifti_spheres_masker._apply_mask_and_get_affinity(
#         seeds=target,
#         niimg=None,
#         radius=radius,
#         allow_overlap=False, 
#         mask_img=brain_mask)

#     sphere_mask = _unmask_3d(
#         X=A.toarray().flatten(), 
#         mask=brain_mask.get_fdata().astype(bool))

#     sphere_mask = Nifti1Image(sphere_mask, brain_mask.affine)

#     sum_vals = np.sum(sphere_mask.get_fdata())

#     if radius == 5:
#         sum_vals = (sum_vals / radius_5_total) * 100
#     elif radius == 10:
#         sum_vals = (sum_vals / radius_10_total) * 100
#     print(sum_vals)

#     while sum_vals < 90:
#         target[0][0] = target[0][0] + 1
#         target[0][1] = target[0][1] - 1
#         target[0][2] = target[0][2] - 1


#         _, A = nifti_spheres_masker._apply_mask_and_get_affinity(
#             seeds=target,
#             niimg=None,
#             radius=radius,
#             allow_overlap=False, 
#             mask_img=brain_mask)

#         sphere_mask = _unmask_3d(
#             X=A.toarray().flatten(), 
#             mask=brain_mask.get_fdata().astype(bool))

#         sphere_mask = Nifti1Image(sphere_mask, brain_mask.affine)

#         sum_vals = np.sum(sphere_mask.get_fdata())

#         if radius == 5:
#             sum_vals = (sum_vals / radius_5_total) * 100
#         elif radius == 10:
#             sum_vals = (sum_vals / radius_10_total) * 100
#     print(target)
#     print(sum_vals)
#     out_df.loc[(out_df.participant_id==subj), "Transformed_Nlin6Asym_Treatment_MNI_X_adj5"] = target[0][0]
#     out_df.loc[(out_df.participant_id==subj), "Transformed_Nlin6Asym_Treatment_MNI_Y_adj5"] = target[0][1]
#     out_df.loc[(out_df.participant_id==subj), "Transformed_Nlin6Asym_Treatment_MNI_Z_adj5"] = target[0][2]

  0%|          | 0/29 [00:00<?, ?it/s]

[array([-39.7742,  47.9738,  31.6835])]
24.691358024691358


  3%|▎         | 1/29 [00:06<03:12,  6.86s/it]

[array([-35.7742,  43.9738,  27.6835])]
90.12345679012346
[array([-47.1543,  43.1123,  28.53  ])]
2.4691358024691357


  3%|▎         | 1/29 [02:14<1:02:56, 134.89s/it]


ValueError: These spheres are empty: [0]

In [13]:
out_df.to_csv(bids_path+'phenotype/coordinates.tsv', index=False, sep="\t")
out_df.head(100)

,participant_id,x,y,z,change_in_MADRS,tier,Transformed_Nlin6Asym_Treatment_MNI_X,Transformed_Nlin6Asym_Treatment_MNI_Y,Transformed_Nlin6Asym_Treatment_MNI_Z,Transformed_Nlin6Asym_Treatment_MNI_X_adj10,Transformed_Nlin6Asym_Treatment_MNI_Y_adj10,Transformed_Nlin6Asym_Treatment_MNI_Z_adj10,Transformed_Nlin6Asym_Treatment_MNI_X_adj5,Transformed_Nlin6Asym_Treatment_MNI_Y_adj5,Transformed_Nlin6Asym_Treatment_MNI_Z_adj5
0,sub-092,-36.0,40.0,38.0,-25.0,2.0,-39.7742,47.97380,31.68350,-34.7742,42.97380,26.68350,-35.7742,43.9738,27.6835
0,sub-093,-42.0,42.0,28.0,-29.0,2.0,-47.1543,43.11230,28.53000,-41.1543,37.11230,22.53000,NaN,NaN,NaN
0,sub-094,-44.0,42.0,28.0,-17.0,1.0,-50.8154,43.27570,11.62400,-46.8154,39.27570,7.62400,NaN,NaN,NaN
0,sub-097,-30.0,42.0,40.0,-5.0,2.0,-30.6853,47.70400,31.00060,-29.6853,46.70400,30.00060,NaN,NaN,NaN
0,sub-098,-36.0,30.0,36.0,-32.0,2.0,-44.6883,40.28540,20.17470,-43.6883,39.28540,19.17470,NaN,NaN,NaN
0,sub-099,-34.0,42.0,28.0,-3.0,2.0,-39.3510,50.18480,22.25630,-36.3510,47.18480,19.25630,NaN,NaN,NaN
0,sub-103,-38.0,34.0,36.0,-12.0,1.0,-43.2435,37.17620,33.29140,-41.2435,35.17620,31.29140,NaN,NaN,NaN
0,sub-105,-40.0,32.0,40.0,-26.0,1.0,-41.4525,34.99170,42.19430,-37.4525,30.99170,38.19430,NaN,NaN,NaN
0,sub-106,-34.0,38.0,32.0,-29.0,1.0,-34.1798,44.90390,25.63820,-34.1798,44.90390,25.63820,NaN,NaN,NaN
0,sub-108,-38.0,0.0,56.0,-23.0,2.0,-37.4534,55.18800,17.20310,-35.4534,53.18800,15.20310,NaN,NaN,NaN


In [ ]:
break

In [ ]:
from nilearn import datasets, plotting
from nilearn.masking import _unmask_3d
from nilearn.maskers import nifti_spheres_masker
import nibabel as nib
from nibabel import Nifti1Image
raidus_5_total = 81
radius_10_total = 515

# let's assume we are in MNI space
#brain_mask = datasets.load_mni152_brain_mask()
brain_mask = nib.load(input_img)
_, A = nifti_spheres_masker._apply_mask_and_get_affinity(
    seeds=target,
    niimg=None,
    radius=2,
    allow_overlap=False, 
    mask_img=brain_mask)

sphere_mask = _unmask_3d(
    X=A.toarray().flatten(), 
    mask=brain_mask.get_fdata().astype(bool))

sphere_mask = Nifti1Image(sphere_mask, brain_mask.affine)

np.sum(sphere_mask.get_fdata())
# nib.save(sphere_mask, "sphere.nii.gz")

# # plot the result to make sure it makes sense
# plotting.plot_roi(sphere_mask)
# plotting.show()

In [ ]:
brain_mask = nib.load(input_img)
_, A = nifti_spheres_masker._apply_mask_and_get_affinity(
    seeds=[(-42, -36, 16)],
    niimg=None,
    radius=1,
    allow_overlap=False, 
    mask_img=brain_mask)

sphere_mask = _unmask_3d(
    X=A.toarray().flatten(), 
    mask=brain_mask.get_fdata().astype(bool))

sphere_mask = Nifti1Image(sphere_mask, brain_mask.affine)

np.sum(sphere_mask.get_fdata())

In [ ]:
from nilearn import plotting
import matplotlib.pyplot as plt
f = plotting.plot_anat(anat_img=input_img, cut_coords=target[0])

In [ ]:
f = plotting.plot_anat(cut_coords=(0, 0, 0))

In [ ]:
from nilearn import plotting
import matplotlib.pyplot as plt

sid = "sub-108"
data = out_df[out_df.participant_id == sid]
coords = [(data.x.values[0], data.y.values[0], data.z.values[0]),
        (data.Transformed_Nlin6Asym_Treatment_MNI_X.values[0],
        data.Transformed_Nlin6Asym_Treatment_MNI_Y.values[0], data.Transformed_Nlin6Asym_Treatment_MNI_Z.values[0])
        ]

f = plotting.plot_anat(anat_img="../../../OCD_gradients/tpl-MNI152NLin6Asym_res-01_T1w.nii.gz", title=sid, cut_coords=coords[0])
f = plotting.plot_anat(anat_img="../../../OCD_gradients/tpl-MNI152NLin6Asym_res-01_T1w.nii.gz", title=sid, cut_coords=coords[1])
plt.show()

In [ ]:
coords[0]

In [ ]:
/home/lukeh/hpcworking/lukeH/projects/QNC_outcomes/data/derivatives/fmriprep_24.0.1/sub-092/ses-pre/sub-092_ses-pre_task-rest_space-MNI152NLin6Asym_res-02_desc-brain_mask.nii.gz